# HBase / HappyBase Tutorial

In [10]:
import string
import random

import happybase

## Connecting via Thrift
The containerized HBase stack includes a trift server to make connecting to HBase simple.  Right now, the port is dynamic, so a `docker-compose ps` is needed to check the external thrift port.  Included in the listing of containers will be a line similar to: `thrift-1         entrypoint.sh thrift             Up      0.0.0.0:32776->9090/tcp`.  This indicats that port 9090 inside the container is mapped to port 32776 on localhost.  Below the connection is opened.

Note that the connection timeout is pretty low, so it might be necessary to reconnect.

In [57]:
connection = happybase.Connection('localhost', port=32886)
try:
    connection.create_table('simple',
                            {'cf1': dict(max_versions=10)})
except:
    print('Table already exists')
    
table = connection.table('simple')

batchsize = 10

rows = range(100000)
cfs = ['a', 'b', 'c', 'd', 'e']
with table.batch(transaction=True) as b:
    for i in range(batchsize):
        rk = 'row{}'.format(random.choice(rows))
        cf1 = 'cf1:a'.encode()
        v = '{}'.format(random.choice(rows)).encode()
        b.put(rk, {cf1:v})

Table already exists


In [37]:
table.families()

{b'cf1': {'block_cache_enabled': False,
  'bloom_filter_nb_hashes': 0,
  'bloom_filter_type': b'NONE',
  'bloom_filter_vector_size': 0,
  'compression': b'NONE',
  'in_memory': False,
  'max_versions': 10,
  'name': b'cf1:',
  'time_to_live': 2147483647}}

In [38]:
table.connection.timeout

In [39]:
table.name

b'simple_table'

In [41]:
table = connection.table('test')

i = 0
for k, d in table.scan():
    print(k,d)
    i += 1
    if i > 10:
        break
    #table.delete(k)

TTransportException: TTransportException(type=4, message='TSocket read 0 bytes')

In [ ]:
table = connection.table('test')


In [23]:
count = 0
for _ in table.scan():
    count += 1
print(count)

94971


In [56]:
connection = happybase.Connection('localhost', port=32886)

table = connection.table('simple')
# Cleaning up without deleting the table
for k, d in table.scan():
    table.delete(k)